In [1]:
import os
import pickle
import numpy as np

In [2]:
path = './data/dense/'
set_my = 'train'
# load visual data
# Training set
v_train_data = open(os.path.join(path, set_my+'Set_visual.p'), 'rb')
# load
visual_train = pickle.load(v_train_data)
# convert to vector
visual_train = np.asarray(visual_train)
# Validation set
v_val_data = open(os.path.join(path, 'valSet_visual.p'), 'rb')
visual_val = pickle.load(v_val_data)
visual_val = np.asarray(visual_val)
# Test Set
v_test_data = open(os.path.join(path, 'testSet_visual.p'), 'rb')
visual_test = pickle.load(v_test_data)
visual_test = np.asarray(visual_test)

# Reading audio data
a_train_data = open(os.path.join(path, set_my+'Set_audio.p'), 'rb')
audio_train = pickle.load(a_train_data)
audio_train = np.asarray(audio_train)

a_val_data = open(os.path.join(path, 'valSet_audio.p'), 'rb')
audio_val = pickle.load(a_val_data)
audio_val = np.asarray(audio_val)

a_test_data = open(os.path.join(path, 'testSet_audio.p'), 'rb')
audio_test = pickle.load(a_test_data)
audio_test = np.asarray(audio_test)

# load Tags
train_target = open(os.path.join(path, set_my+'Set_target.p'), 'rb')
target_train = pickle.load(train_target)
target_train = np.asarray(target_train)

val_target = open(os.path.join(path, 'valSet_target.p'), 'rb')
target_val = pickle.load(val_target)
target_val = np.asarray(target_val)

test_target = open(os.path.join(path, 'testSet_target.p'), 'rb')
target_test = pickle.load(test_target)
target_test = np.asarray(target_test)

print('visual_train:', visual_train.shape)
print('visual_val:', visual_val.shape)
print('visual_test:', visual_test.shape)
print('audio_train:', audio_train.shape)
print('audio_val:', audio_val.shape)
print('audio_test:', audio_test.shape)
print('target_train:', target_train.shape)
print('target_val:', target_val.shape)
print('target_test:', target_test.shape)
print('data load done')
print('-----------------')

visual_train: (3339, 10, 1920)
visual_val: (402, 10, 1920)
visual_test: (402, 10, 1920)
audio_train: (3339, 10, 12, 8, 512)
audio_val: (402, 10, 12, 8, 512)
audio_test: (402, 10, 12, 8, 512)
target_train: (3339, 28)
target_val: (402, 28)
target_test: (402, 28)
data load done
-----------------


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [8]:
def create_visual_lstm_model(input_shape, num_classes):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.TimeDistributed(layers.Dense(128, activation='relu')),  # Apply Dense to the input at each time step
        layers.LSTM(64),  # LSTM layer to process time series
        layers.Dense(num_classes, activation='softmax')  # Classification Output
    ])
    return model

#visual input
#10 time steps, 1280-dimensional features per step
input_shape = (10, 1920)  
num_classes = 28  # classification number

visual_lstm_model = create_visual_lstm_model(input_shape, num_classes)
visual_lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
visual_lstm_model.fit(visual_train, target_train, validation_data=(visual_val, target_val), epochs=50, batch_size=64)

# Evaluating the Model
visual_lstm_model.evaluate(visual_test, target_test)


Epoch 1/50
53/53 [==============================] - 1s 9ms/step - loss: 2.2215 - accuracy: 0.4322 - val_loss: 1.4017 - val_accuracy: 0.5995
Epoch 2/50
53/53 [==============================] - 0s 4ms/step - loss: 1.1712 - accuracy: 0.6765 - val_loss: 1.0821 - val_accuracy: 0.7040
Epoch 3/50
53/53 [==============================] - 0s 4ms/step - loss: 0.9017 - accuracy: 0.7329 - val_loss: 0.9239 - val_accuracy: 0.7214
Epoch 4/50
53/53 [==============================] - 0s 6ms/step - loss: 0.7286 - accuracy: 0.7817 - val_loss: 0.7926 - val_accuracy: 0.7662
Epoch 5/50
53/53 [==============================] - 0s 6ms/step - loss: 0.6371 - accuracy: 0.8074 - val_loss: 0.8182 - val_accuracy: 0.7338
Epoch 6/50
53/53 [==============================] - 0s 6ms/step - loss: 0.5495 - accuracy: 0.8374 - val_loss: 0.7552 - val_accuracy: 0.7711
Epoch 7/50
53/53 [==============================] - 0s 6ms/step - loss: 0.4598 - accuracy: 0.8640 - val_loss: 0.7447 - val_accuracy: 0.7463
Epoch 8/50
53/53 [==

[1.230528473854065, 0.7313432693481445]

In [9]:
def create_audio_model_adjusted(input_shape, num_classes):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        #Convolutional Layer
        layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu', padding='same')), 
        layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
        layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu', padding='same')),
        layers.TimeDistributed(layers.Flatten()),
        # Sequence Processing Layer
        layers.LSTM(64),
        # Classification layer
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

input_shape = (10, 12, 8, 512)  # audio input
num_classes = 28 #Number of categories
# Create and compile the model
adjusted_model = create_audio_model_adjusted(input_shape, num_classes)
adjusted_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training and evaluating the model
adjusted_model.fit(audio_train, target_train, validation_data=(audio_val, target_val), epochs=50, batch_size=64)

adjusted_model.evaluate(audio_test, target_test) 

Epoch 1/50
53/53 [==============================] - 4s 50ms/step - loss: 2.6339 - accuracy: 0.2186 - val_loss: 2.0899 - val_accuracy: 0.3507
Epoch 2/50
53/53 [==============================] - 2s 36ms/step - loss: 1.7140 - accuracy: 0.4864 - val_loss: 1.4415 - val_accuracy: 0.5597
Epoch 3/50
53/53 [==============================] - 2s 36ms/step - loss: 1.2632 - accuracy: 0.6280 - val_loss: 1.1489 - val_accuracy: 0.6667
Epoch 4/50
53/53 [==============================] - 2s 36ms/step - loss: 1.0162 - accuracy: 0.6954 - val_loss: 1.0324 - val_accuracy: 0.7065
Epoch 5/50
53/53 [==============================] - 2s 35ms/step - loss: 0.8506 - accuracy: 0.7433 - val_loss: 1.0488 - val_accuracy: 0.6841
Epoch 6/50
53/53 [==============================] - 2s 36ms/step - loss: 0.7775 - accuracy: 0.7649 - val_loss: 0.9997 - val_accuracy: 0.6891
Epoch 7/50
53/53 [==============================] - 2s 36ms/step - loss: 0.6648 - accuracy: 0.7972 - val_loss: 0.8987 - val_accuracy: 0.7338
Epoch 8/50
53

[1.3025453090667725, 0.7263681888580322]

In [10]:
from tensorflow.keras import layers, models, Input

def create_multimodal_model(visual_input_shape, audio_input_shape, num_classes):
    # Visual sub-model
    visual_input = Input(shape=visual_input_shape)
    visual_model = layers.TimeDistributed(layers.Dense(256, activation='relu'))(visual_input)
    visual_model = layers.LSTM(64)(visual_model)

    # Audio sub-model
    audio_input = Input(shape=audio_input_shape)
    audio_model = layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))(audio_input)
    audio_model = layers.TimeDistributed(layers.MaxPooling2D((2, 2)))(audio_model)
    audio_model = layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))(audio_model)
    audio_model = layers.TimeDistributed(layers.Flatten())(audio_model)
    audio_model = layers.LSTM(64)(audio_model)

    # Fusion of visual and audio features
    combined = layers.concatenate([visual_model, audio_model])

    # Fully connected classifier
    combined = layers.Dense(128, activation='relu')(combined)
    output = layers.Dense(num_classes, activation='softmax')(combined)

    # define model
    model = models.Model(inputs=[visual_input, audio_input], outputs=output)
    return model


visual_input_shape = (10, 1920)  # visual input
audio_input_shape = (10, 12, 8, 512)  # audio input
num_classes = 28  # category

# Creating and compiling a multimodal model
multimodal_model = create_multimodal_model(visual_input_shape, audio_input_shape, num_classes)
multimodal_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
multimodal_model.fit([visual_train, audio_train], target_train, validation_data=([visual_val, audio_val], target_val), epochs=50, batch_size=64)

# Evaluating the Model
multimodal_model.evaluate([visual_test, audio_test], target_test)


Epoch 1/50
53/53 [==============================] - 5s 58ms/step - loss: 2.1638 - accuracy: 0.4043 - val_loss: 1.0309 - val_accuracy: 0.6965
Epoch 2/50
53/53 [==============================] - 2s 36ms/step - loss: 0.8080 - accuracy: 0.7550 - val_loss: 0.7019 - val_accuracy: 0.7786
Epoch 3/50
53/53 [==============================] - 2s 35ms/step - loss: 0.5614 - accuracy: 0.8239 - val_loss: 0.5866 - val_accuracy: 0.8234
Epoch 4/50
53/53 [==============================] - 2s 36ms/step - loss: 0.4116 - accuracy: 0.8688 - val_loss: 0.5567 - val_accuracy: 0.8134
Epoch 5/50
53/53 [==============================] - 2s 36ms/step - loss: 0.3162 - accuracy: 0.9000 - val_loss: 0.5200 - val_accuracy: 0.8333
Epoch 6/50
53/53 [==============================] - 2s 36ms/step - loss: 0.2670 - accuracy: 0.9179 - val_loss: 0.4476 - val_accuracy: 0.8483
Epoch 7/50
53/53 [==============================] - 2s 36ms/step - loss: 0.1757 - accuracy: 0.9476 - val_loss: 0.4361 - val_accuracy: 0.8507
Epoch 8/50
53

[0.7059783935546875, 0.8557214140892029]